In [ ]:
import pathlib
import re
import collections
import functools
import difflib
import operator
import json
import requests
import time
import editdistance
import random

datadir = pathlib.Path("/home/david/pro/scc/data")

%run utils.py
import similarity

In [ ]:
relevant = tuple(str(year) for year in (1845, 1850, 1860, 1880, 1885))

In [ ]:
with (datadir / ".." / "links" / "linkede_personer.csv").open("r", encoding="latin1") as fd:
    print(next(fd))
    links = {year: [] for year in relevant}
    linked_ids = {year: set() for year in relevant}
    for line in fd:
        split = [part.lower() for part in line.strip().split("|")]
        for year in relevant:
            if year in split[1]:
                links[year].append(split)
                linked_ids[year].add(tuple(split[2:4]))
                break

In [ ]:
for year, matched in links.items():
    print(year, len(matched))

In [ ]:
data = {}
for fn in sorted(datadir.glob("lc_*.csv")):
    year = re.search(r"\d{4}", fn.name).group(0)
    print(year)
    data[year] = {} # year -> kip,løb -> row
    with fn.open("r", encoding="UTF-8") as fd:
        next(fd)
        current_set = linked_ids[year]
        print("Now:", len(current_set))
        for line in fd:
            row = line.strip().split("|")
            kip, løb = row[-2:]
            løb = løb.split(",")[0]
            pair = (kip,løb)
            if pair in current_set:
                current_set.remove(pair)
        print("After:", len(current_set))
    break
            #row[8] = normalize_more(row[8])

In [ ]:
linked_ids